# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [17]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2024-05-22 03:23:19,539 - 140136843099008 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "London",
    'tentative_date': "2024-06-01",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [19]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in London that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in London that meets the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "venues for conferences in London"} 


Search results: Title: The 15 Best Conference Venues for Hire in London
Link: https://hirespace.com/GB/London/Conference-Venues
Snippet: Conference Venues In London Ideas · One Great George Street - image. Westminster · Barbican Centre - image. City of London · Baker Street. The Landmark London.
---
Title: The 16 Best Conference Venues for Hire in London - Tagvenue
Link: https://www.tagvenue.com/hire/conference-venues/london
Snippet: Top Conference Venues in London · Elizabeth Room · Summer Daily Delegate Rate · The Davy Mine · Club Room · Main Hall · Theatre · Tea + Coffee · Entire Venue.
---
Title: Conferen

This is the agent final answer: After researching various conference venues in London, I have found several options that could be suitable for the Tech Innovation Conference:

1. One Great George Street, Westminster
   - Capacity: 240 people
   - Room hire includes a technician and standard audio visual equipment
   - Ideal for large conferences, presentations, seminars, and product launches

2. Barbican Centre, City of London
   - Capacity: 1,943 people
   - Fixed seating arena with tiered seating and excellent acoustics
   - Suitable for larger conferences with superb technical resources

3. The Landmark London, Baker Street
   - Capacity: 600 people
   - Offers quintessential London charm in a luxurious setting
   - Ideal for corporate events and business presentations

4. Conway Hall, Camden
   - Capacity: 400 people
   - Retains period features and fully accessible by wheelchair
   - Equipped for recording or livestreaming events and hosting hybrid physical/virtual conferences

5.

Thought: 
After re-evaluating my options, I need to make a final decision on the venue for the Tech Innovation Conference.

Action: Delegate work to co-worker
Action Input: {"coworker": "Logistics Manager", "task": "Book Central Hall Westminster for the Tech Innovation Conference", "context": "Central Hall Westminster can accommodate up to 2,000 attendees and offers a cozy ambiance for AGMs, Conferences, and special occasions with cutting-edge audio-visual systems."}

> Entering new CrewAgentExecutor chain...
I need to find information about booking Central Hall Westminster for the Tech Innovation Conference.

Action: Read website content
Action Input: {"website_url": "centralhall.co.uk"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'centralhall.co.uk': No scheme supplied. Perhaps you meant https://centralhall.co.uk?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a

Final Answer: 
Central Hall Westminster is one of the largest conference and events venues in Central London, offering 23 varied rooms and spaces to accommodate events of all sizes. They can cater to intimate functions of up to 60 people and large occasions for up to 500 people, as well as high-profile events of over 2,000 people with ease. The Grade II* listed building has a rich history of hosting events of cultural significance and is known for its distinctive architecture and grandeur. The venue provides high-quality audio-visual equipment, bespoke catering options, and many rooms with eye-catching period features. With over a century of history, Central Hall Westminster has strong links with various sectors and has welcomed illustrious figures through its doors. To enquire about booking Central Hall Westminster for the Tech Innovation Conference, you can contact their sales team at 020 7222 8010.

> Finished chain.
 

Central Hall Westminster is one of the largest conference and e

Thought: I need to gather more information about the Tech Innovation Conference to effectively promote it and engage potential attendees.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference London details"}I need to find a venue for an event with 500 participants on June 1, 2024. Central Hall Westminster seems like a good option based on the capacity and availability.

Action: Read website content
Action Input: {"website_url": "https://www.c-h-w.com/"} 

Events venue & meeting rooms hire | Central Hall Westminster HomePlan Your Event◎Why book CHWEvent Spaces◎Rooms 1000 to 2000 people◎Great HallLecture Hall and LibraryRooms 250 to 500 people◎Lecture HallLibraryAldersgateRooms up to 150 people◎George ThomasRobert PerksDonald EnglishWilliam SangsterRooms up to 60 people◎President’sBroadbentJohn TudorDinsdale YoungMaurice BarnettSyndicate roomsEvent Types◎ConferencesMeetingsExhibitions and galleriesGraduationsConcertsSeminarsBroadcasting and TV productio

Final Answer: Based on the website content from Central Hall Westminster, it is a suitable venue for an event with 500 participants on June 1, 2024. The venue offers 23 varied rooms and spaces to accommodate events of all sizes, from intimate functions of up to 60 people to large occasions for up to 500 people. Additionally, they can cater for high-profile events of over 2,000 people with ease. The Grade II* listed building has a rich history of hosting events of cultural significance and is well-equipped with audiovisual equipment and catering services. Overall, Central Hall Westminster seems like the perfect fit for the event with 500 participants on June 1, 2024.

> Finished chain.
 

Based on the website content from Central Hall Westminster, it is a suitable venue for an event with 500 participants on June 1, 2024. The venue offers 23 varied rooms and spaces to accommodate events of all sizes, from intimate functions of up to 60 people to large occasions for up to 500 people. Addi

Exception in thread Thread-12 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/u

Thought: I now know the final answer
Final Answer: 

# Tech Innovation Conference Promotion Report

## Marketing Activities

### Search Results:
1. **Title:** Technology & Innovation Summit North America - Forrester
   - **Link:** [Forrester Tech Summit](https://www.forrester.com/event/technology-innovation-north-america/)
   - **Description:** Unleash The Power Of Tech, Talent, And AI.

2. **Title:** Gartner Tech Growth & Innovation Conference 2024 in Grapevine, TX
   - **Link:** [Gartner Tech Growth](https://www.gartner.com/en/conferences/na/tech-growth-us)
   - **Description:** Program for the 2025 GartnerTech Growth & Innovation Conference is underway.

3. **Title:** CES - The Most Powerful Tech Event in the World
   - **Link:** [CES Tech Event](https://www.ces.tech/)
   - **Description:** CES® is the proving ground for breakthrough technologies and global innovators.

### London Specific Search Results:
1. **Title:** Tech and Innovation Summit 2024 - techUK
   - **Link:** [techUK 

- Display the generated `venue_details.json` file.

In [20]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': "Central Hall Westminster, Storey's Gate, Westminster, London SW1H "
            '9NH',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'Central Hall Westminster'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [21]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement Report

### Social Media Engagement Strategies:
- Post engaging content leading up to the event
- Create a hashtag for the event
- Use targeted ads to reach more people
- Leverage industry influencers
- Run contests and giveaways
- Respond to comments and messages
- Utilize visual content to captivate the audience
- Select the best times to post on social media
- Share behind-the-scenes details
- Host a countdown to the event

### Additional Marketing Strategies:
- Start the marketing campaign early to build anticipation
- Create compelling marketing content
- Engage with potential attendees through various channels
- Utilize email marketing to reach out to target audience segments
- Collaborate with sponsors and partners for increased visibility
- Utilize innovative brand boosting ideas to attract more attendees

By implementing these marketing strategies and engaging potential attendees through social media and other channels, the Tech Innovation Conference can maximize event exposure and achieve the goal of engaging at least 500 potential attendees.